## <span style="color: blue;">1️⃣ <b><u>Import Libraries</u></b> </span>

In [37]:
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime as dt
import numpy as np
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import GOOG
from pandas_datareader import data as web

## <span style="color: green;">2️⃣ <b><u>Pull Data From Yahoo Finance</u></b></span>

In [153]:
# Define the function to get stock data
def get_stock_data(ticker, start_date, end_date, interval):
    df = yf.download(ticker, start=start_date, end=end_date, interval=interval)
    df.dropna(inplace=True)
    
    # Drop the 'Adj Close' column if it exists
    if 'Adj Close' in df.columns:
        df.drop(columns=['Adj Close'], inplace=True)
    
    # Ensure single-level column names
    df.columns = df.columns.get_level_values(0)  # Flatten columns
    df.columns.name = None  # Remove the name attribute from columns
    
    return df

## 3️⃣ <span style="color: red;"><b><u>Trading Strategies</u></b></span>

### <b> 1. </b> Moving Average Ribbon Strategy [Parv]

In [54]:
def moving_average_ribbon_strategy(df):
    short_window = 10
    long_window = 50
    df['Short_MA'] = df['Close'].rolling(window=short_window, min_periods=1).mean()
    df['Long_MA'] = df['Close'].rolling(window=long_window, min_periods=1).mean()
    
    df['Signal'] = 0
    df['Signal'][short_window:] = np.where(df['Short_MA'][short_window:] > df['Long_MA'][short_window:], 1, 0)
    
    return df

### <b> 2. </b> SuperTrend Strategy [Parv]

In [52]:
def supertrend_strategy(df, atr_period=7, multiplier=3):
    high_low = df['High'] - df['Low']
    high_close = np.abs(df['High'] - df['Close'].shift())
    low_close = np.abs(df['Low'] - df['Close'].shift())
    tr = pd.DataFrame([high_low, high_close, low_close]).max(axis=0)
    atr = tr.rolling(atr_period).mean()
    
    df['Upper_Band'] = (df['High'] + df['Low']) / 2 + (multiplier * atr)
    df['Lower_Band'] = (df['High'] + df['Low']) / 2 - (multiplier * atr)
    df['Supertrend'] = np.nan
    
    for i in range(1, len(df)):
        if df['Close'][i] > df['Upper_Band'][i - 1]:
            df['Supertrend'][i] = df['Lower_Band'][i]
        elif df['Close'][i] < df['Lower_Band'][i - 1]:
            df['Supertrend'][i] = df['Upper_Band'][i]
        else:
            df['Supertrend'][i] = df['Supertrend'][i - 1]
    
    df['Signal'] = np.where(df['Close'] > df['Supertrend'], 1, 0)
    
    return df

### <b> 3. </b> Volatility Breakout Strategy [Parv]

In [ ]:
def volatility_breakout_strategy(df, lookback=20, multiplier=2):
    df['High_Max'] = df['High'].rolling(lookback).max()
    df['Low_Min'] = df['Low'].rolling(lookback).min()
    df['Range'] = df['High_Max'] - df['Low_Min']
    
    df['Buy_Level'] = df['Open'] + (multiplier * df['Range'].shift())
    df['Sell_Level'] = df['Open'] - (multiplier * df['Range'].shift())
    
    df['Signal'] = 0
    df['Signal'] = np.where(df['Close'] > df['Buy_Level'], 1, df['Signal'])
    df['Signal'] = np.where(df['Close'] < df['Sell_Level'], -1, df['Signal'])
    
    return df

### <b> 4. </b> Money Flow Index (MFI) Strategy [Anand]

In [61]:
def mfi_strategy(df, period=14):
    typical_price = (df['High'] + df['Low'] + df['Close']) / 3
    money_flow = typical_price * df['Volume']
    
    positive_flow = money_flow.copy()
    negative_flow = money_flow.copy()
    
    positive_flow[typical_price < typical_price.shift(1)] = 0
    negative_flow[typical_price > typical_price.shift(1)] = 0
    
    positive_mf = positive_flow.rolling(period).sum()
    negative_mf = negative_flow.rolling(period).sum()
    
    mfi = 100 - (100 / (1 + (positive_mf / negative_mf)))
    df['MFI'] = mfi
    
    df['Signal'] = 0
    df['Signal'] = np.where(df['MFI'] < 20, 1, df['Signal'])
    df['Signal'] = np.where(df['MFI'] > 80, -1, df['Signal'])
    
    return df

### <b> 5. </b> Volume Price Trend (VPT) Strategy [Anand]

In [64]:
def vpt_strategy(df):
    vpt = ((df['Close'].pct_change() + 1).cumprod() - 1) * df['Volume']
    df['VPT'] = vpt.cumsum()
    
    df['Signal'] = 0
    df['Signal'] = np.where(df['VPT'] > df['VPT'].shift(), 1, 0)
    
    return df

### <b> 6. </b> Heikin-Ashi Candlesticks Strategy [Anand]

In [67]:
def heikin_ashi_strategy(df):
    ha_df = df.copy()
    ha_df['Close'] = (df['Open'] + df['High'] + df['Low'] + df['Close']) / 4
    ha_df['Open'] = (df['Open'].shift() + df['Close'].shift()) / 2
    ha_df['High'] = df[['Open', 'Close', 'High']].max(axis=1)
    ha_df['Low'] = df[['Open', 'Close', 'Low']].min(axis=1)
    
    df['HA_Close'] = ha_df['Close']
    df['HA_Open'] = ha_df['Open']
    
    df['Signal'] = 0
    df['Signal'] = np.where(df['HA_Close'] > df['HA_Open'], 1, df['Signal'])
    df['Signal'] = np.where(df['HA_Close'] < df['HA_Open'], -1, df['Signal'])
    
    return df

### <b> 7. </b> Renko Chart Strategy [Neel]

In [73]:
def renko_strategy(df, brick_size=2):
    renko_df = df.copy()
    renko_df['Brick'] = (renko_df['Close'] // brick_size) * brick_size
    
    renko_df['Uptrend'] = renko_df['Brick'] > renko_df['Brick'].shift()
    renko_df['Downtrend'] = renko_df['Brick'] < renko_df['Brick'].shift()
    
    df['Signal'] = 0
    df['Signal'] = np.where(renko_df['Uptrend'], 1, df['Signal'])
    df['Signal'] = np.where(renko_df['Downtrend'], -1, df['Signal'])
    
    return df

### <b> 8. </b> Mean Reversion with Bollinger Bands and RSI Strategy [Neel]

In [79]:
def bollinger_bands_rsi_strategy(df, bb_period=20, rsi_period=14):
    df['MA20'] = df['Close'].rolling(window=bb_period).mean()
    df['BB_Upper'] = df['MA20'] + 2 * df['Close'].rolling(window=bb_period).std()
    df['BB_Lower'] = df['MA20'] - 2 * df['Close'].rolling(window=bb_period).std()
    
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=rsi_period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=rsi_period).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    df['Signal'] = 0
    df['Signal'] = np.where((df['Close'] < df['BB_Lower']) & (df['RSI'] < 30), 1, df['Signal'])
    df['Signal'] = np.where((df['Close'] > df['BB_Upper']) & (df['RSI'] > 70), -1, df['Signal'])
    
    return df

### <b> 9. </b> Dual Moving Average Strategy [Neel]

In [251]:
class DualMovingAverage(Strategy):
    short_window = 50
    long_window = 200

    def init(self):
        # Calculate short and long moving averages
        close_prices = self.data.Close
        print(close_prices)
        self.short_ma = pd.Series(close_prices).rolling(window=self.short_window, min_periods=self.short_window).mean().values
        self.long_ma = pd.Series(close_prices).rolling(window=self.long_window, min_periods=self.long_window).mean().values
        print(self.short_ma)
        print(self.long_ma)

    def next(self):
        #print("Trying to Transact")
        # Ensure that both moving averages are valid before proceeding 
        if np.isnan(self.short_ma[-1]) or np.isnan(self.long_ma[-1]):
            print("Ignoring because of NA")
            return
        # Trading logic based on the crossover of moving averages
        if crossover(self.short_ma, self.long_ma):
            print("Buying")
            self.buy()
        elif crossover(self.long_ma, self.short_ma):
            print("Selling")
            self.sell()
            
# Example usage with sample data
ticker = 'AAPL'
start_date = '2015-01-01'
end_date = '2021-01-01'

# Download historical data for backtesting
df = get_stock_data(ticker, start_date, end_date, '1d')

# Drop the 'Adj Close' column if it exists
if 'Adj Close' in df.columns:
    df.drop(columns=['Adj Close'], inplace=True)

# Ensure single-level column names
df.columns = df.columns.get_level_values(0)  # Flatten columns
df.columns.name = None  # Remove the name attribute from columns

# Reset index to include Date column
df.reset_index(inplace=True)
df.set_index('Date', inplace=True)

# Ensure columns are in correct order
df = df[['Open', 'High', 'Low', 'Close', 'Volume']]

df.head()
# Calculate short and long moving averages
#short_window = 50
#long_window = 200

#df['Short_MA'] = df['Close'].rolling(window=short_window, min_periods=1).mean()
#df['Long_MA'] = df['Close'].rolling(window=long_window, min_periods=1).mean()

# Display the DataFrame with the new columns
#print(df.head())

# Backtest the strategy
#bt = Backtest(df, DualMovingAverage, cash=10000, commission=.002)
#stats = bt.run()
#print(stats)
#bt.plot()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Date,,,,,
2015-01-02,27.847500,27.860001,26.837500,27.332500,212818400
2015-01-05,27.072500,27.162500,26.352501,26.562500,257142000
2015-01-06,26.635000,26.857500,26.157499,26.565001,263188400
2015-01-07,26.799999,27.049999,26.674999,26.937500,160423600
2015-01-08,27.307501,28.037500,27.174999,27.972500,237458000


In [159]:
# Drop the 'Adj Close' column if it exists
if 'Adj Close' in df.columns:
    df.drop(columns=['Adj Close'], inplace=True)

# Ensure single-level column names
df.columns = df.columns.get_level_values(0)  # Flatten columns
df.columns.name = None  # Remove the name attribute from columns

# Reset index to include Date column
df.reset_index(inplace=True)
df.set_index('Date', inplace=True)

# Ensure columns are in correct order
df = df[['Open', 'High', 'Low', 'Close', 'Volume']]

# Display the DataFrame
print(df.head())


                 Open       High        Low      Close     Volume
Date                                                             
2015-01-02  27.847500  27.860001  26.837500  27.332500  212818400
2015-01-05  27.072500  27.162500  26.352501  26.562500  257142000
2015-01-06  26.635000  26.857500  26.157499  26.565001  263188400
2015-01-07  26.799999  27.049999  26.674999  26.937500  160423600
2015-01-08  27.307501  28.037500  27.174999  27.972500  237458000


## <span style="color: purple;">4️⃣ <b><u>Backtesting</u></b></span>

In [21]:
# Function to backtest a given trading strategy
def backtest_strategy(strategy, df):
    bt = Backtest(df, strategy, cash=10000, commission=.002)
    stats = bt.run()
    bt.plot()
    return stats

## <span style="color: brown;">5️⃣ <u><b>Comparing with TD Mutual Fund</b></u></span>

In [40]:
# Function to compare trading strategy with a mutual fund
def compare_with_mutual_fund(strategy_results, mutual_fund_data):
    combined_data = pd.concat([strategy_results['_equity_curve']['Equity'], mutual_fund_data], axis=1)
    combined_data.columns = ['Strategy Equity', 'Mutual Fund']
    combined_data.plot(figsize=(12, 6), title='Strategy vs Mutual Fund Comparison')
    plt.ylabel('Equity')
    plt.show()

# Function to load mutual fund data
def get_mutual_fund_data(ticker, start_date, end_date, interval='1d'):
    return get_stock_data(ticker, start_date, end_date, interval)['Close']

# Example usage with a placeholder trading strategy
def trading_strategy(df):
    # Placeholder strategy - Buy and hold
    df['Signal'] = 1
    return df